In [11]:
import geopandas as gpd
import numpy as np
import pandas as pd

In [100]:
results = pd.read_csv('../data/bevoelkerung/pop_2021.csv', sep=',', encoding='Latin-1')

In [110]:
def read_pop(results):
    results = pd.read_csv('../data/bevoelkerung/pop_2021.csv', sep=',', encoding='Latin-1')

    results = results.drop(['Jahr', 'Geschlecht'], axis=1)
    results = results.rename(columns={'Kanton (-) / Bezirk (>>) / Gemeinde (......)' :'GMDNAME'})
    df = results.loc[~results['GMDNAME'].str.startswith('>>')]
    df = df.loc[~df['GMDNAME'].str.startswith('-')]
    df['GMDNAME'] = df['GMDNAME'].str[6:]
    df['GMDNAME'] = df['GMDNAME'].str[:4]
    df = df.drop(index=0)
    df = df.reset_index(drop=True)
    df = df.drop(index=len(df)-1)
    df['GMDNAME'] = df['GMDNAME'].astype(int)
    df = df.rename(columns={'GMDNAME': 'BFSNR'})
    df = df.rename(columns={'Bestand am 1. Januar': 'POP_TOTAL'})

    return df


def pop_clean(population):
    rename_cols = {
        'BFS': 'BFSNR',
        'Total': 'POP_TOTAL',
        'Total.1': 'POP_CH_TOTAL',
        'Total.2': 'POP_FOREIGN_TOTAL'
    }

    drop_cols = ['GMDNAME', 'Mann', 'Frau',
                 'Mann.1', 'Frau.1', 'Mann.2', 'Frau.2']

    population = population.rename(columns=rename_cols)
    population = population.drop(columns=drop_cols)
    population = population.reindex(
        columns=['BFSNR', 'POP_TOTAL', 'POP_CH_TOTAL', 'POP_FOREIGN_TOTAL'])

    return population

In [111]:
df_pop = read_pop(results)

In [112]:
test = read_pop(results)

In [113]:
test.dtypes

BFSNR        int64
POP_TOTAL    int64
dtype: object

In [114]:
master = gpd.read_file('../export/master/master_acc_bike_pop_street.geojson')

In [115]:
master = master.merge(test, on='BFSNR', how='left')

In [121]:
master['ACC_100K'] = master['ACCIDENTS'] / master['POP_TOTAL'] * 100000

In [124]:
# calculate mean of ACC_100K per KTNR
master['ACC_100K_MEAN'] = master.groupby('KTNR')['ACC_100K'].transform('mean')

In [131]:
master['YES_IN_PER'] = master['YES_IN_PER'].astype(float)

ValueError: could not convert string to float: '...'

In [130]:
master.groupby('KTNR')['ACC_100K'].mean()
#master.groupby('KTNR')['YES_IN_PER'].mean()

TypeError: Could not convert 72.5572.9573.9870.8499999999999974.3666.7900000000000176.267.8671.370.0971.5674.2863.7271.9754.3153.2859.5377.3499999999999958.1773.6655.578.3770.9273.0100000000000164.572.260.5962.275.2900000000000163.157.8353.1350.9668.4771.8371.6172.1471.4875.9865.2600000000000165.5957.5363.3356.9366.5665.1261.4261.5353.1565.0965.0699999999999963.2269.8870.5968.8661.8360.363.1361.5559.0561.0164.8660.4562.0961.8164.8369.6266.0966.2900000000000166.0100000000000176.5860.9559.9964.8870.8199999999999955.7961.7153.5462.4171.6270.4300000000000149.8667.3464.4266.0699999999999970.5400000000000172.5268.1800000000000171.555.7366.1164.0569.5270.0263.3370.7900000000000168.8968.564.763.169.3462.6973.0968.0563.1872.6367.9263.0267.2600000000000169.2869.4569.7099999999999972.5999999999999963.1164.4160.9665.1500000000000171.3959.0869.3466.8967.876.2399999999999962.387073.1676.6763.7164.2900000000000163.8167.9599999999999967.5675.9265.9899999999999969.1270.7900000000000156.4273.5670.900000

In [116]:
#master.to_file('../export/master/00_master_acc_bike_pop_street.geojson', driver='GeoJSON')

In [79]:
#master.groupby('BZNR').count().sort_values(by='BFSNR', ascending=False)

In [80]:
def percentage(df_in):
    df = df_in.copy()
    
    # Change types etc.
    df = df.replace('...', 0)
    df = df.fillna(0)
    df['YES_IN_PER'] = df['YES_IN_PER'].astype(float)
    df['BIKE_STREET_PER'] = df['BIKE_STREET_PER'].astype(float)

    # Change percentages to decimals
    df['YES_IN_PER'] = df['YES_IN_PER'] / 100
    df['BIKE_STREET_PER'] = df['BIKE_STREET_PER'] / 100

    return df

In [81]:
def calc_ratio(df_in):
    df = df_in
    
    max = df['BIKE_STREET_PER'].max()
    df['MAX_RATIO'] = ((df['BIKE_STREET_PER'] * 100) / max)
    

    return df

In [82]:
def calc_corr_bz(df_in, master):
    df = df_in
    
    #df = df.replace('...', 0)
    #df['YES_IN_PER'] = df['YES_IN_PER'].astype(float) /100
    #df['MAX_RATIO'] = df['MAX_RATIO']/100

    correlations = df.groupby('BZNR')['YES_IN_PER', 'MAX_RATIO'].corr()

    df = correlations.reset_index()
    df_selected = df[df.index % 2 == 0]
    # create a new DataFrame with only the 'BZNR' and 'MAX_RATIO' columns
    df_new = df_selected[['BZNR', 'MAX_RATIO']].copy()
    df_new = df_new.rename(columns={'MAX_RATIO': 'CORR'})

    # merge master and de_new on 'BZNR' and only keep the 'CORR' column from df_new


    df_out = master.merge(df_new, on='BZNR', how='left') # only merge the column 'CORR' from df_new

    df_out = df_out.drop(columns=['MAX_RATIO'])

    
    return df_out
    

In [83]:
def calc_corr_ch(df_in):
    df = df_in
    # calculate correlation between 'YES_IN_PER' and 'BIKE_STREET_PER'
    correlation = df.corr(method='pearson', min_periods=1)['YES_IN_PER']['BIKE_STREET_PER']
    
    return correlation

In [84]:
master = percentage(master)

In [85]:
corr = calc_corr_bz(calc_ratio(master), master)

/var/folders/lb/9kd2fxwx6z79bw37hynhftf00000gn/T/ipykernel_63846/1190528316.py:8: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  correlations = df.groupby('BZNR')['YES_IN_PER', 'MAX_RATIO'].corr()


In [87]:
corr.dtypes

BFSNR                   int64
GMDNAME                object
ELIGIBLE_VOTERS        object
TOT_VOTES              object
PART_PERCENT           object
VALID_VOTES            object
YES                    object
NO                     object
YES_IN_PER            float64
BZNR                    int64
KTNR                    int64
E_CNTR                  int64
N_CNTR                  int64
ACCIDENTS             float64
SUM_FREQ              float64
SUM_LENGTH            float64
POP_TOTAL             float64
POP_CH_TOTAL          float64
POP_FOREIGN_TOTAL     float64
LENGTH                float64
COUNT                 float64
BIKE_STREET_PER       float64
geometry             geometry
CORR                  float64
dtype: object

In [88]:
df_new = corr.merge(df_pop, on='BFSNR', how='left')

In [96]:
df_new['POP'] = df_new['POP'].astype(float)

In [97]:
#print all rows where POP is NaN
df_new[df_new['POP'].isna()]

,BFSNR,GMDNAME,ELIGIBLE_VOTERS,TOT_VOTES,PART_PERCENT,VALID_VOTES,YES,NO,YES_IN_PER,BZNR,...,SUM_LENGTH,POP_TOTAL,POP_CH_TOTAL,POP_FOREIGN_TOTAL,LENGTH,COUNT,BIKE_STREET_PER,geometry,CORR,POP
22,291,Andelfingen,1509,768,50.89,760,539,221,0.709200,102,...,15177.461400,0.0,0.0,0.0,208085.409052,1651.0,0.07294,"POLYGON ((2692625.000 1273555.000, 2693440.000...",0.649441,NaN
1029,3112,Schwende-Rüte,4169,1274,30.56,1247,812,435,0.651200,1600,...,15141.368590,0.0,0.0,0.0,514301.815486,2879.0,0.02944,"POLYGON ((2755973.000 1246109.000, 2755989.000...",0.237245,NaN
1089,3396,Neckertal,2921,1023,35.02,1015,618,397,0.608900,1727,...,22747.940380,0.0,0.0,0.0,550000.627720,3086.0,0.04136,"POLYGON ((2729900.000 1249967.000, 2730270.000...",0.414952,NaN
1306,4186,ch-Ueken,1654,585,35.36880290205563,578,358,220,0.619377,1906,...,42.071263,0.0,0.0,0.0,128994.895641,928.0,0.00033,"POLYGON ((2645822.000 1260791.000, 2646072.000...",-0.197197,NaN
1318,4185,Böztal,1800,669,37.17,666,421,245,0.632100,1906,...,6625.194396,0.0,0.0,0.0,248141.930868,1472.0,0.02670,"POLYGON ((2650713.000 1263186.000, 2651327.000...",-0.197197,NaN
1403,4324,Zurzach,4194,1528,36.43,1516,1020,496,0.672800,1911,...,18752.816790,0.0,0.0,0.0,331121.056092,2640.0,0.05663,"MULTIPOLYGON (((2674332.758 1269005.404, 26737...",0.565019,NaN
1568,5240,Val Mara,1796,750,41.76,729,535,194,0.733900,2105,...,4955.421067,0.0,0.0,0.0,68391.973271,567.0,0.07246,"POLYGON ((2719735.000 1089445.000, 2720525.000...",-0.353728,NaN
1889,5892,Blonay - Saint-Légier,6910,2954,42.75,2936,2551,385,0.868900,2230,...,138.312654,0.0,0.0,0.0,268823.874622,2188.0,0.00051,"POLYGON ((2560975.000 1150495.000, 2561245.000...",-0.031796,NaN
2115,6811,eux-Lugenz,301,102,33.88704318936877,100,72,28,0.720000,2603,...,0.000000,0.0,0.0,0.0,63515.996723,373.0,0.00000,"POLYGON ((2576583.000 1258798.000, 2576638.000...",0.205687,NaN


In [10]:
calc_corr_ch(corr)

/var/folders/lb/9kd2fxwx6z79bw37hynhftf00000gn/T/ipykernel_63697/1683115305.py:4: FutureWarning: The default value of numeric_only in GeoDataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlation = df.corr(method='pearson', min_periods=1)['YES_IN_PER']['BIKE_STREET_PER']


0.06199858610606

In [56]:
#corr.to_file('../tmp/corr.shp', driver='ESRI Shapefile')

/var/folders/lb/9kd2fxwx6z79bw37hynhftf00000gn/T/ipykernel_62284/2728858944.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  corr.to_file('../tmp/corr.shp', driver='ESRI Shapefile')


# UNUSED

In [ ]:
def calc_bike_street_ratio(df_in):
    df = df_in.copy()
    df['BIKE_STREET_PER'] = np.round((df['SUM_LENGTH'] / df['LENGTH']) , 2)

    return df

In [ ]:
def calc_ratio(df_in):
    df = df_in.copy()
    #replace entries with '...' with 0
    df['YES_IN_PER'] = df['YES_IN_PER'].replace('...', 0)
    #change type of 'YES_IN_PER' to float
    df['YES_IN_PER'] = df['YES_IN_PER'].astype(float)
    #fill na values with 0
    df['YES_IN_PER'] = df['YES_IN_PER'].fillna(0)
    #Fill na values with 0 for all columns
    df = df.fillna(0)
    #df['DIV'] = (df['BIKE_STREET_PER'] / df['YES_IN_PER'])
    #df['ADD'] = (df['BIKE_STREET_PER'] + df['YES_IN_PER'])
    #df['SUB'] = (df['BIKE_STREET_PER'] - df['YES_IN_PER'])
    #df['MUL'] = (df['BIKE_STREET_PER']/100) * (df['YES_IN_PER']/100)
    ## PER DIFF subtract value A from value B, divide the result by the average of the two values, and then multiply by 100 to get a percentage
    #df['PER_DIFF_Y'] = (df['YES_IN_PER'] - df['BIKE_STREET_PER']) / ((df['BIKE_STREET_PER'] + df['YES_IN_PER'])/2) * 100
    #df['PER_DIFF_B'] = (df['BIKE_STREET_PER'] - df['YES_IN_PER']) / ((df['BIKE_STREET_PER'] + df['YES_IN_PER'])/2) * 100


    return df